In [1]:
import pandas as pd
import quandl #quandl is a platform from where we have a number of datasets we can load a data, we can buy a paid data and also upload our own data.
import math
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LinearRegression

In [2]:
df = quandl.get('WIKI/GOOGL')

In [3]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [4]:
#Features

In [5]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

In [6]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Low'] * 100.0

In [7]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

In [8]:
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]

In [9]:
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0
2004-08-20,54.322689,8.537313,7.227007,22834300.0
2004-08-23,54.869377,4.062357,-1.227880,18256100.0
2004-08-24,52.597363,7.753210,-5.726357,15247300.0
2004-08-25,53.164113,3.966115,1.183658,9188600.0


In [10]:
#Labels

In [11]:
forecast_col = 'Adj. Close' 

In [12]:
df.fillna(-99999, inplace=True)

In [13]:
df.isnull().sum()

Adj. Close     0
HL_PCT         0
PCT_change     0
Adj. Volume    0
dtype: int64

In [14]:
forecast_out = int(math.ceil(0.01*len(df )))

In [15]:
forecast_out

35

In [16]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [17]:
df['label']

Date
2004-08-19    69.078238
2004-08-20    67.839414
2004-08-23    68.912727
2004-08-24    70.668146
2004-08-25    71.219849
                ...    
2018-03-21          NaN
2018-03-22          NaN
2018-03-23          NaN
2018-03-26          NaN
2018-03-27          NaN
Name: label, Length: 3424, dtype: float64

In [18]:
df.head()
#our labels are adj. close value 10 days in future.

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849


In [19]:
df.tail()
#there are nan value of label at last because we take a only 0.01 percent of the total length. 

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2018-03-21,1094.00,1.976619,0.130884,1990515.0,NaN
2018-03-22,1053.15,3.265882,-2.487014,3418154.0,NaN
2018-03-23,1026.55,4.089299,-2.360729,2413517.0,NaN
2018-03-26,1054.09,4.818025,0.332191,3272409.0,NaN
2018-03-27,1006.94,6.707965,-5.353887,2940957.0,NaN


In [20]:
df.dropna(inplace=True)

In [21]:
df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2018-01-30,1177.37,1.142604,-0.029718,1792602.0,1094.00
2018-01-31,1182.22,1.213207,-0.134312,1643877.0,1053.15
2018-02-01,1181.59,1.547000,0.476195,2774967.0,1026.55
2018-02-02,1119.20,1.811604,-0.729098,5798880.0,1054.09
2018-02-05,1068.76,5.512236,-2.893850,3742469.0,1006.94


In [22]:
X = np.array(df.drop(columns = ['label'],axis = 1))
Y = np.array(df['label'])

In [23]:
X = preprocessing.scale(X) #normalised

In [24]:
print(len(X))
print(len(Y))

3389
3389


In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [26]:
clf = LinearRegression()
clf.fit(X_train,Y_train)

LinearRegression()

In [27]:
accuracy = clf.score(X_test,Y_test)

In [28]:
accuracy

0.9786009162743898